In [ ]:
# e-commerce dataset - https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce

from langchain_core.prompts import PromptTemplate
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model="llama3.2")

table_description = """The first table is 'customers'.
It contains information about customers.
The table includes columns:
- customer_id (str) - unique identifier for each customer,
- customer_unique_id (str) - unique key associated with each customer (different from customer_id),
- customer_zip_code_prefix (str) - prefix of the customer's zip code,
- customer_city (str) - city where the customer resides,
- customer_state (str) - state where the customer resides, represented by a two-letter code.

The second table is 'products'.
It includes information about products.
The table includes columns:
- product_id (str) - unique identifier for each product,
- product_category_name (str) - name of the product category in Portuguese,
- product_name_length (int) - number of characters in the product name,
- product_description_length (int) - number of characters in the product description,
- product_photos_qty (int) - number of photos associated with the product,
- product_weight_g (int) - weight of the product in grams,
- product_length_cm (int) - length of the product in centimeters,
- product_height_cm (int) - height of the product in centimeters,
- product_width_cm (int) - width of the product in centimeters.

The third table is 'orders'.
It contains information about orders placed by customers.
The table includes columns:
- order_id (str) - unique identifier for each order,
- customer_id (str) - unique identifier for the customer who placed the order,
- order_status (str) - status of the order (e.g., delivered, invoiced),
- order_purchase_timestamp (datetime) - date and time when the order was placed,
- order_approved_at (datetime) - date and time when the order was approved,
- order_delivered_carrier_date (datetime) - date and time when the order was handed over to the carrier,
- order_delivered_customer_date (datetime) - date and time when the order was delivered to the customer,
- order_estimated_delivery_date (datetime) - estimated date and time for delivery of the order.
"""

template_question_to_sql = """[INST] You are given data tables in postgres.

[DATA_DESCRIPTION]
{data_description}
[\\DATA_DESCRIPTION]

Your task is to identify what data are required to answer a user's question and
to generate a valid SQL query for postgres that will include data required to answer a user's question.

Answer is a JSON: {{"sql_query":  valid_sql_query}}. Don't add anything else.
When merging two tables add a table identifier.
If you can't provide an answer based on the tables, say so.

[EXAMPLE]For example:
User question: "What is the total number of orders placed by customers from the state of SP in 2018?"
Expected answer in JSON: {{"sql_query": "SELECT COUNT(o.order_id) AS total_orders 
FROM ecommerce.orders AS o 
LEFT JOIN ecommerce.customers AS c ON o.customer_id = c.customer_id 
WHERE o.order_purchase_timestamp >= '2018-01-01' 
AND o.order_purchase_timestamp < '2019-01-01' 
AND c.customer_state = 'SP'"}}
[\\EXAMPLE]

User question: {text}
[\\INST]"""


template_sql_to_insight = """[INST] Your task is to answer user questions ONLY based on the provided data frame and the SQL query used to generate the table.

[EXAMPLE]For example:
User question: "What is the total number of orders placed by customers from the state of SP in 2018?"
SQL query: "SELECT COUNT(o.order_id) AS total_orders 
FROM ecommerce.orders AS o 
LEFT JOIN ecommerce.customers AS c ON o.customer_id = c.customer_id 
WHERE o.order_purchase_timestamp >= '2018-01-01' 
AND o.order_purchase_timestamp < '2019-01-01' 
AND c.customer_state = 'SP'""
Data frame: "	total_orders
            0   23871"
Answer: "The total number of orders placed by customers from the state of SP in 2018 is 23871."
[\\EXAMPLE]

Answer should be specific and precise, don't add anything else!
If you can't answer the question based on the provided data, say so, don't try to guess!

User question: {text}
SQL query: {sql_query}
Data frame: {table}
[\\INST]"""

def text2sql(question,
             question_to_sql_prompt = template_question_to_sql,
             df_to_insight_prompt = template_sql_to_insight,
             data_description = table_description):
    import json

    # get dataframe
    prompt_sql = PromptTemplate(
        template=question_to_sql_prompt,
        input_variables=[
            "text",
            "data_description",
        ]
    )
    # print(prompt_sql)
    

question = "What is the total number of orders placed by customers from the state of SP in 2017?"
text2sql(question)

input_variables=['data_description', 'text'] input_types={} partial_variables={} template='[INST] You are given data tables in postgres.\n\n[DATA_DESCRIPTION]\n{data_description}\n[\\DATA_DESCRIPTION]\n\nYour task is to identify what data are required to answer a user\'s question and\nto generate a valid SQL query for postgres that will include data required to answer a user\'s question.\n\nAnswer is a JSON: {{"sql_query":  valid_sql_query}}. Don\'t add anything else.\nWhen merging two tables add a table identifier.\nIf you can\'t provide an answer based on the tables, say so.\n\n[EXAMPLE]For example:\nUser question: "What is the total number of orders placed by customers from the state of SP in 2018?"\nExpected answer in JSON: {{"sql_query": "SELECT COUNT(o.order_id) AS total_orders \nFROM ecommerce.orders AS o \nLEFT JOIN ecommerce.customers AS c ON o.customer_id = c.customer_id \nWHERE o.order_purchase_timestamp >= \'2018-01-01\' \nAND o.order_purchase_timestamp < \'2019-01-01\' \nA